In [ ]:
import pandas as pd
import numpy as np 
import datetime as dt
import matplotlib.pyplot as plt
import mplfinance as mpf
import warnings
import math
import os
# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
# Read the CSV file and parse dates
df = pd.read_csv("Banknifty_Spot_data.csv", parse_dates=['datetime'])
offset = 0
# Select the desired columns
df = df[['datetime', 'open', 'high', 'low', 'close']]


df = df.rename(columns={'datetime': 'Date'})
df.set_index('Date', inplace=True)



In [ ]:
# # Read your CSV data into a DataFrame
# df = pd.read_csv("BANKNIFTY24MARFUT.csv", header=None, names=['Date', 'Time', 'Tick', 'Column1', 'Column2'])

# # Convert 'DATE' and 'TIME' columns to datetime and combine them into a single timestamp column
# df['Date'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'])

# # Set 'TIMESTAMP' as the index
# df.set_index('Date', inplace=True)

# # Resample the data to 5-second intervals
# resampled_df = df.resample('5S').agg({
#     'Tick': ['first', 'max', 'min', 'last']  # Open, High, Low, Close
# })

# # Flatten the multi-level column index
# resampled_df.columns = ['open', 'high', 'low', 'close']

# # Drop rows with any missing values
# resampled_df.dropna(inplace=True)

# # Reset index
# resampled_df.reset_index(inplace=True)

# # Display the resampled DataFrame
# resampled_df



In [ ]:

# # Set 'Date' column as index
# resampled_df.set_index('Date', inplace=True)

# df = resampled_df[resampled_df.index.month == 2]
# df = df[-10000:]

In [ ]:
df

In [ ]:
def Supertrend(df, atr_period, multiplier):
    high = df['high']
    low = df['low']
    close = df['close']
    
    # Calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    atr = true_range.ewm(alpha=1/atr_period, min_periods=atr_period).mean() 
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # Upperband and lowerband calculation
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # Print lengths before the loop
    print("Lengths before loop:")
    print("close:", len(close))
    print("final_upperband:", len(final_upperband))
    print("final_lowerband:", len(final_lowerband))
    
    # Initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    # Initialize buy and sell signals
    buy_signal = [0] * len(df)
    sell_signal = [0] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i - 1
        
        # Check data length before accessing elements
        if curr >= len(close) or prev >= len(close):
            break
        
        # If current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
            buy_signal[curr] = 1
        # If current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
            sell_signal[curr] = -1
        # Else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # Adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # To remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    # Print lengths after the loop
    print("\nLengths after loop:")
    print("close:", len(close))
    print("final_upperband:", len(final_upperband))
    print("final_lowerband:", len(final_lowerband))
    
    # Assign 'Sell_Put' and 'Sell_Call' values based on buy and sell signals
    signal_values = ['' for _ in range(len(df))]
    for i in range(len(df)):
        if buy_signal[i] == 1:
            signal_values[i] = 'PE'
        elif sell_signal[i] == -1:
            signal_values[i] = 'CE'
    # Print lengths of buy and sell signals
    print("Length of buy_signal:", len(buy_signal))
    print("Length of sell_signal:", len(sell_signal))
    
    return pd.DataFrame({
        'Supertrend': supertrend,
        'Final Lowerband': final_lowerband,
        'Final Upperband': final_upperband,
        'Buy_Signal': buy_signal,
        'Sell_Signal': sell_signal,
        'Signal_Value': signal_values
    }, index=df.index)


In [ ]:
# Apply SuperTrend indicator to the DataFrame
atr_period = 30
atr_multiplier = 10
supertrend_df = Supertrend(df, atr_period, atr_multiplier)


In [ ]:
# Join the SuperTrend results with the original DataFrame, specifying suffixes
df = df.join(supertrend_df, rsuffix='_supertrend')

In [ ]:
# Add a Volume column with zeros
df['volume'] = 0

# Define the SuperTrend indicator
supertrend_lowerband = mpf.make_addplot(df['Final Lowerband'], color='green', alpha=0.3)
supertrend_upperband = mpf.make_addplot(df['Final Upperband'], color='red', alpha=0.3)
supertrend_fill = mpf.make_addplot(df[['Final Lowerband', 'Final Upperband']], color='gray', alpha=0.2)

In [ ]:

# Define buy and sell signals
buy_signal = df[df['Buy_Signal'] == 1]['close']
sell_signal = df[df['Sell_Signal'] == -1]['close']

In [ ]:
# Filter the DataFrame based on the condition
buy_signal_indices = np.where(df['Buy_Signal'] != 0)[0]
sell_signal_indices = np.where(df['Sell_Signal'] != 0)[0]

# Merge the indices
merged_indices = np.concatenate((buy_signal_indices, sell_signal_indices))

# Filtered DataFrame
filtered_df = df.iloc[merged_indices]

# Sort the DataFrame based on the index (assuming 'Date' is the index)
filtered_df = filtered_df.sort_index()

# Reset the index of filtered_df
filtered_df.reset_index(inplace=True)

# Shift the 'Date' column by -1 to get the exit time
filtered_df['Entry_time'] = filtered_df['Date']
filtered_df['Exit_time'] = filtered_df['Date'].shift(-1)
filtered_df

In [ ]:
filtered_df.info()

In [ ]:
# Calculate Spot
# Round 'close' column to the nearest 100
rounded_close = filtered_df['close'].round(-2)

# Adjust 'close_spot' based on 'Signal_Value'
filtered_df['close_spot'] = rounded_close.copy()  # Create a copy of rounded_close
filtered_df.loc[filtered_df['Signal_Value'] == 'CE', 'close_spot'] += offset
filtered_df.loc[filtered_df['Signal_Value'] == 'PE', 'close_spot'] -= offset

# Round 'Final Upperband' or 'Final Lowerband' column to the nearest 100
filtered_df['Supertrend_spot'] = np.where(filtered_df['Final Upperband'].notnull(),
                                          filtered_df['Final Upperband'].round(-2),
                                          filtered_df['Final Lowerband'].round(-2))
# Display the modified DataFrame
filtered_df

In [ ]:
filtered_df.set_index('Date', inplace=True)

# Extract year, month, and day from the index
year = filtered_df.index.year.astype(str).str[-2:]
month = filtered_df.index.month.astype(str).str.zfill(2)
day = filtered_df.index.day.astype(str).str.zfill(2)

# Convert close_spot to integer and then string
close_spot = filtered_df['close_spot'].astype(int).astype(str)

Supertrend_spot = filtered_df['Supertrend_spot'].astype(int).astype(str)

# Extract CE or PE from Signal_Value column
strike_type = filtered_df['Signal_Value'].str[-2:]

In [ ]:
# Strategy 1 -------> Selling Supertrend_spot Strike

#filtered_df['sell_strike_symbol'] = 'BANKNIFTY' + year + month + day + Supertrend_spot + strike_type


# Strategy 2 -------> Selling close_spot Strike
filtered_df['sell_strike_symbol'] = 'BANKNIFTY' + year + month + day + close_spot + strike_type



In [ ]:
# Drop the last row
filtered_df = filtered_df.drop(filtered_df.index[-1])
filtered_df

In [ ]:
import os

def find_premium_at_time(file_path, timestamp):
    try:
        data = pd.read_csv(file_path, header=None, usecols=[0, 1, 2], names=['Date', 'Time', 'Tick'])
        data['Timestamp'] = pd.to_datetime(data['Date'].astype(str) + ' ' + data['Time'], errors='coerce')
        idx = data['Timestamp'].sub(timestamp).abs().idxmin() 
        premium = data.loc[idx, 'Tick']
        return premium
    except FileNotFoundError:
        return None

def process_data(filtered_df, directory):
    for index, row in filtered_df.iterrows():
        sell_strike_symbol = row['sell_strike_symbol']
        entry_time = pd.Timestamp(row['Entry_time']) + pd.Timedelta(minutes=1)
        exit_time = pd.Timestamp(row['Exit_time']) + pd.Timedelta(minutes=1)
        
        # Construct file path
        file_path = os.path.join(directory, f"{sell_strike_symbol}.csv")
        
        try:
            # Find sell premium at entry time
            sell_premium = find_premium_at_time(file_path, entry_time)
            filtered_df.at[index, 'sell_premium'] = sell_premium
            
            # Find buy premium at exit time
            buy_premium = find_premium_at_time(file_path, exit_time)
            filtered_df.at[index, 'buy_premium'] = buy_premium
        except Exception as e:
            print(f"An error occurred for file '{file_path}': {e}")
            continue  # Skip to the next iteration if an error occurs
        
    return filtered_df

# Example usage
directory = r'C:\Users\user\Desktop\Workspace\BANKNIFTY Options'

print("Before calling find_premium_at_time function")
processed_df = process_data(filtered_df, directory)
print("After calling find_premium_at_time function")
print(processed_df)



In [ ]:
processed_df['P&L'] = processed_df['sell_premium'] - processed_df['buy_premium']

In [ ]:
report_df = processed_df[['Entry_time','Exit_time','sell_strike_symbol','sell_premium','buy_premium','P&L']]
report_df.dropna(inplace=True)
report_df['cumm'] = report_df['P&L'].cumsum()


In [ ]:
report_df

In [ ]:
report_df['cumm'].plot()

In [ ]:
report_df.to_excel("BNF_Supertrend_ATM.xlsx", index=False)


In [ ]:

# # Set the figure size
# plt.figure(figsize=(16, 10))  # Adjust the width and height as needed

# # Plot candlestick data with SuperTrend indicators
# mpf.plot(df, type='candle', style='charles', volume=True, title='Candlestick Chart with SuperTrend Indicator',
#          ylabel='Price', ylabel_lower='Volume', addplot=[supertrend_lowerband, supertrend_upperband, supertrend_fill])


In [ ]:


# # Set the figure size
# plt.figure(figsize=(16, 10))  # Adjust the width and height as needed

# # Plot candlestick data with SuperTrend indicators
# mpf.plot(df, type='candle', style='charles', volume=True, title='Candlestick Chart with SuperTrend Indicator',
#          ylabel='Price', ylabel_lower='Volume', addplot=[supertrend_lowerband, supertrend_upperband, supertrend_fill])




In [ ]:
# # Plot buy signals if they exist
# if not buy_signal.empty:
#     plt.scatter(buy_signal.index, buy_signal, marker='^', color='green', s=100, label='Buy Signal')

# # Plot sell signals if they exist
# if not sell_signal.empty:
#     plt.scatter(sell_signal.index, sell_signal, marker='v', color='red', s=100, label='Sell Signal')

# # Show legend
# plt.legend()

# # Show the combined plot
# plt.show()
